# Importance sampling: toy example with parameters, sampling one-by-one

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid

import matplotlib.pyplot as plt

Functions for generation and visualization of the image batches

In [2]:
def visualize_img_batch(batch):
    '''Visualizes image batch'''
    grid = make_grid(batch, nrow=8, padding=1, normalize=False, range=None, scale_each=False, pad_value=0)
    plt.imshow(grid.permute(1,2,0))
    plt.show()

In [3]:
def random_image_data(size=(32768, 3, 1, 1), ratio=0.5):
    '''Makes a random image batch of size (batch_size, height, width, channels) 
    with black to white ratio of value ratio
    '''
    idx = torch.randperm(size[0])[:int(ratio*size[0])]
    image_batch = torch.zeros(size) + 0.2 # to make light gray
    image_batch[idx] = 1 - 0.2 # to make light gray 
    return image_batch

The dataset class

In [4]:
class BlackWhiteDataset(Dataset):
    '''The dataloader for the black and white images'''
    def __init__(self, weight_network):
        self.dataset = random_image_data()
        
        self.weight_network = weight_network

    def __len__(self):
        return len(self.dataset)
    
    def accept_sample(self, weight_network, img):
        # Returns True if the image is accepted, False if rejected
        weight = weight_network(img)
        return bool(list(torch.utils.data.sampler.WeightedRandomSampler([1-weight, weight], 1))[0])
    
    def __getitem__(self, idx):
        # Random permutation on the dataset order (is this equivalent to uniform sampling?)
        all_idx = torch.randperm(len(dataset))[:100]
        
        # Loop through the samples and return once accepted
        for i in all_idx:
            accept = self.accept_sample(self.weight_network, self.dataset[i])
            if accept:
                return self.dataset[i]
        return self.dataset[all_idx[0]] # return a random one if no sample found

The weight network with parameters

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 1)
        self.fc2 = nn.Linear(1, 1)

    def forward(self, x):
        h1 = torch.sigmoid(self.fc1(x.view(-1, 3)))
        out = torch.sigmoid(self.fc2(h1))
        return out

weight_network = Net()

In [6]:
dataset = BlackWhiteDataset(weight_network)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0)

In [7]:
example_img = torch.cat((torch.Tensor([0.2, 0.2, 0.2]), torch.Tensor([0.8, 0.8, 0.8])))
w = weight_network(example_img)
print(w)

tensor([[0.3670],
        [0.3659]], grad_fn=<SigmoidBackward>)


With Helge's objective function

In [11]:
weight_network = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(weight_network.parameters(), lr=0.1)
dataset = BlackWhiteDataset(weight_network)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0)

for i, data in enumerate(dataloader):
    labels = data.mean(1).view(-1, 1)

    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = weight_network(data)
    
    # Ground truth:
    ground_truth = data.mean(1).view(-1, 1) 
    
    loss = ((data.mean(1).view(-1, 1)-0.5) * (outputs/outputs.detach()).view(-1, 1)).sum()
    
    loss.backward()
    optimizer.step()

    # print statistics
    if i % 50 == 0:
        w = weight_network(example_img)
        print('step', i, 'loss: ', loss.item())
        print('outputs:', w[0].item(), w[1].item(), 'ground_truth', example_img[0].item(), example_img[3].item())
        print()

step 0 loss:  1.8000000715255737
outputs: 0.7183206081390381 0.7107142210006714 ground_truth 0.20000000298023224 0.800000011920929

step 50 loss:  -7.199999809265137
outputs: 0.8067004680633545 0.3645223081111908 ground_truth 0.20000000298023224 0.800000011920929

step 100 loss:  -18.0
outputs: 0.973055362701416 0.04105329141020775 ground_truth 0.20000000298023224 0.800000011920929

step 150 loss:  -17.39999771118164
outputs: 0.9871178269386292 0.01744568906724453 ground_truth 0.20000000298023224 0.800000011920929

step 200 loss:  -18.599998474121094
outputs: 0.9919471144676208 0.01144480798393488 ground_truth 0.20000000298023224 0.800000011920929

step 250 loss:  -18.600000381469727
outputs: 0.9944936633110046 0.009124783799052238 ground_truth 0.20000000298023224 0.800000011920929

step 300 loss:  -19.19999885559082
outputs: 0.995403528213501 0.006913210730999708 ground_truth 0.20000000298023224 0.800000011920929

step 350 loss:  -19.19999885559082
outputs: 0.996434211730957 0.0060458

The network learns to give importance 1 to dark grey images and importance 0 to light grey images.

In [ ]:
visualize_img_batch(data)
print('light grey = 0.2, dark grey = 0.8')